# process_cmip6_files.ipynb
Post-processing of CMIP6 data that have been downloaded from ESGF:
1. Regrid to regular lon/lat grid (using CDO).

B. S. Grandey, 2022

In [1]:
! date

Fri Jan 28 13:47:53 +08 2022


In [2]:
import contextlib
import pathlib
from cdo import Cdo, CDOException

In [3]:
cdo = Cdo()

print(f'CDO version: {cdo.version()}')
print(f'cdo.py bindings version: {cdo.__version__()}')

CDO version: 2.0.3
cdo.py bindings version: 1.5.4


## Base paths

In [4]:
# CMIP6 input data downloaded from ESGF (organised as <variable>/<source_id>_<member_id>/<instance_id>)
in_base = pathlib.Path('~/Data/p22b/CMIP6/').expanduser()
# Output data
out_base = pathlib.Path('~/Data/p22c/CMIP6/').expanduser()
out_base.mkdir(exist_ok=True)

## Function regrid available data (using CDO) for a given variable, table_id, source-member pair, and experiment
Notes regarding available interpolation methods, after testing CDO's gencon2, gencon, and genbic on Omon zos data:
1. For unstructured grids, gencon/gencon2 works but not genbic ("Bilinear/bicubic interpolation doesn't support unstructured source grids!").
2. For curvilinear grids, genbic works. gencon/gencon2 mostly works, but fails for some grids (with most common error being "Source grid cell corner coordinates missing!").
3. For lonlat grids, all methods work.

Based on the above observations, one possibility would be to try gencon2 (2nd order conservative). If gencon2 fails, then use gencon (1st order conservative). If gencon2 and gencon both fail, then use genbic (bicubic). In case all three fail, include gennn (nearest neighbour) as a final fallback.

In [5]:
def regrid_using_cdo(variable='zos',
                     table_id='Omon',
                     source_member='ACCESS-CM2_r1i1p1f1',
                     experiment='historical',
                     stat_str='-yearmean',  # calculate statistical values after regridding?
                     preferred_methods=['gencon2', 'gencon', 'genbic', 'gennn'],  # highest priority first
                     force=False,  # overwrite previously produced files?
                    ):
    
    print(f'---- {variable} {table_id} {source_member} {experiment} {stat_str} ----')
    
    # List of pre-existing files skipped (assuming force=False)
    existing_list = []
    # List of output files written
    out_list = []
    
    # Search for available input directories, then identify most appropriate directory
    in_dirs = list(in_base.glob(f'{variable}/{source_member}/CMIP6.*.{experiment}.*.{table_id}.{variable}.*'))
    in_dirs = sorted(in_dirs)
    # Case: no input directories
    if len(in_dirs) == 0:
        print('No suitable input data found')
        return None  # exit (if no input directories)
    # Case: one suitable input directory
    elif len(in_dirs) == 1:
        in_dir = in_dirs[0]
        print(f'1 input dir: {in_dir.name} ({len(list(in_dir.glob("*.nc")))} nc file(s))')
    # Case: more than one suitable input directory
    else:
        print(f'{len(in_dirs)} input dirs:')
        for d in in_dirs:
            print(f'{d.name} ({len(list(d.glob("*.nc")))} nc file(s))')
        # Select most recently dated version
        vmax = 0  # initialise "max" version as 0
        for d in in_dirs:
            vmax = max(vmax, int(d.name[-8:]))
        for d in in_dirs:
            if str(vmax) not in d.name:
                in_dirs.remove(d)
        # After version selection, does only one input dir remain?
        if len(in_dirs) == 1:
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (most recent) ')
        # Prefer data that has already been remapped
        else:
            for d in in_dirs:
                if '.gn.' in d.name:
                    in_dirs.remove(d)
            in_dir = in_dirs[0]
            print(f'Using {in_dir.name} (non-native grid)')
    
    # Identify input data files
    in_fns = sorted(list(in_dir.glob('*.nc')))
    # Input grid type
    try:
        with contextlib.redirect_stdout(None):  # suppress CDO error message details (if any)
            grid_str = [s for s in cdo.sinfo(input=f'{in_fns[0]}') if 'points' in s][0]
        in_grid_type = grid_str.split(':')[1].strip()
        in_grid_points = grid_str.split(':')[-1].strip()
        print(f'Input data grid type is {in_grid_type} ({in_grid_points})')
    except CDOException:  # if Globus transfer is in progress, incomplete files may be encountered
        print(f'Failed to read {in_fns[0].name} - is it currently being transferred?')
        return None  # exit (if incomplete file encountered)
    # Output data directory
    if stat_str:
        out_dir = out_base.joinpath(f'regrid_{stat_str.strip("-")}/{variable}/{source_member}/{in_dir.name}')
    else:
        out_dir = out_base.joinpath(f'regrid/{variable}/{source_member}/{in_dir.name}')
    out_dir.mkdir(parents=True, exist_ok=True)
    
    # If points=1 (e.g. due to already being a global mean), then no need to regrid
    if in_grid_points == 'points=1':
        print('No need to regrid')
        # Loop over in_fns (if points=1)
        for in_fn in in_fns:
            # Output filename (if points=1)
            if stat_str:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.{stat_str.strip("-")}.nc')
            else:
                out_fn = out_dir.joinpath(f'{in_fn.stem}.1d.nc')
            # Do not overwrite, unless force=True (if points=1)
            if out_fn.exists() and not force:
                existing_list.append(out_fn.name)
                continue
            # Write output file (if points=1)
            cdo.selname(variable, input=f'{stat_str} {in_fn}', output=f'{out_fn}')
            if out_fn.exists():
                out_list.append(out_fn.name)
            else:
                print(f'Failed to write {out_fn.name}')
        # Print summary of existing files skipped (if points=1)
        if len(existing_list) == 1:
            print(f'Skipped existing file: {existing_list[0]}')
        elif len(existing_list) > 1:
            print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
        # Print summary of files written (if points=1)
        if len(out_list) == 1:
            print(f'Written {out_list[0]}')
        elif len(out_list) > 1:
            print(f'Written {len(out_list)} files, inc {out_list[0]}')
        return out_list  # exit (if points=1)
    
    # Weights data directory
    weights_dir = out_base.joinpath(f'regrid_weights/{variable}/{source_member}/{in_dir.name}')
    weights_dir.mkdir(parents=True, exist_ok=True)
    # Generate regridding weights file
    for method in preferred_methods:
        print(f'Trying {method}')
        weights_fn = weights_dir.joinpath(f'{in_fns[0].stem}.{method}_weights.nc')
        # For unstructured grids, do not use gencon2
        if method == 'gencon2' and in_grid_type == 'unstructured':
            print(f'Skipping gencon2 for unstructured grid')
            continue
        # If weights file already exists, use pre-existing file (unless force=True)
        if weights_fn.exists() and not force:
            print(f'{weights_fn.name} already exists')
            break  # if weights file exists, no need to try other methods
        # Otherwise, try calculating weights and writing file
        else:
            try:
                with contextlib.redirect_stdout(None):  # suppress CDO error message details
                    if method == 'gencon2':
                        cdo_res = cdo.gencon2('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gencon':
                        cdo_res = cdo.gencon('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')          
                    elif method == 'genbic':
                        cdo_res = cdo.genbic('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    elif method == 'gennn':
                        cdo_res = cdo.gennn('global_1', input=f'{in_fns[0]}', output=f'{weights_fn}')
                    else:
                        continue
                if weights_fn.exists():
                    print(f'Written {weights_fn.name}')
                    break  # if file written successfully, then no need to try other methods
            except CDOException:
                print(f'Failed to write {weights_fn.name}')
    
    # Loop over input data files and regrid
    print(f'{len(in_fns)} input data file(s) to regrid')
    for in_fn in in_fns:
        # Output data filename
        if stat_str:
            out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.{stat_str.strip("-")}.nc')
        else:
            out_fn = out_dir.joinpath(f'{in_fn.stem}.{method[3:]}.nc')
        # If output file already exists, do not overwrite (unless force=True)
        if out_fn.exists() and not force:
            existing_list.append(out_fn.name)
            continue
        # Regrid using CDO
        cdo_res = cdo.sellonlatbox(0, 360, -90, 90,  # shift longitudes
                                   input=f'{stat_str} -selname,{variable} -remap,global_1,{weights_fn} {in_fn}',
                                   output=f'{out_fn}')
        # Check if written
        if out_fn.exists():
            out_list.append(out_fn.name)
        else:
            print(f'Failed to write {out_fn.name}')
    
    # Print summary of existing files skipped
    if len(existing_list) == 1:
        print(f'Skipped existing file: {existing_list[0]}')
    elif len(existing_list) > 1:
        print(f'Skipped {len(existing_list)} existing files, inc {existing_list[0]}')
    # Print summary of files written (if points=1)
    if len(out_list) == 1:
        print(f'Written {out_list[0]}')
    elif len(out_list) > 1:
        print(f'Written {len(out_list)} files, inc {out_list[0]}')
    
    return out_list  # exit

## Apply function(s) to variables and experiments of interest

In [6]:
# For variables and experiments of interest, apply regridding function to all available source-member pairs
# Loop over variables of interest
for variable, table_id in [('zostoga', 'Omon'),
                           ('tas', 'Amon')]:
    # Identify and loop over available source-member pairs
    source_member_pairs = sorted([d.name for d in in_base.glob(f'{variable}/[!.]*_*')])
    for source_member in source_member_pairs:
        # Loop over experiments of interest
        for experiment in ['historical', ]:
            # Regrid
            temp = regrid_using_cdo(variable=variable,
                                    table_id=table_id,
                                    source_member=source_member,
                                    experiment=experiment,
                                    force=True)
        print('')  # empty line between source-member pairs

---- zostoga Omon ACCESS-CM2_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historical.r1i1p1f1.Omon.zostoga.gn.v20191108 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Written zostoga_Omon_ACCESS-CM2_historical_r1i1p1f1_gn_185001-201412.yearmean.nc

---- zostoga Omon ACCESS-ESM1-5_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CSIRO.ACCESS-ESM1-5.historical.r1i1p1f1.Omon.zostoga.gn.v20191115 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Written zostoga_Omon_ACCESS-ESM1-5_historical_r1i1p1f1_gn_185001-201412.yearmean.nc

---- zostoga Omon BCC-CSM2-MR_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.BCC.BCC-CSM2-MR.historical.r1i1p1f1.Omon.zostoga.gn.v20181126 (1 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Written zostoga_Omon_BCC-CSM2-MR_historical_r1i1p1f1_gn_185001-201412.yearmean.nc

---- zostoga Omon BCC-ESM1_r1i1p1f1 historical -yearmean

Input data grid type is generic (points=1)
No need to regrid
Written 9 files, inc zostoga_Omon_MPI-ESM-1-2-HAM_historical_r1i1p1f1_gn_185001-186912.yearmean.nc

---- zostoga Omon MPI-ESM1-2-HR_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.MPI-M.MPI-ESM1-2-HR.historical.r1i1p1f1.Omon.zostoga.gn.v20190710 (33 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Written 33 files, inc zostoga_Omon_MPI-ESM1-2-HR_historical_r1i1p1f1_gn_185001-185412.yearmean.nc

---- zostoga Omon MPI-ESM1-2-LR_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.MPI-M.MPI-ESM1-2-LR.historical.r1i1p1f1.Omon.zostoga.gn.v20190710 (9 nc file(s))
Input data grid type is generic (points=1)
No need to regrid
Written 9 files, inc zostoga_Omon_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_185001-186912.yearmean.nc

---- zostoga Omon MRI-ESM2-0_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.MRI.MRI-ESM2-0.historical.r1i1p1f1.Omon.zostoga.gm.v20191205 (1 nc file(s))
Input data 

Input data grid type is gaussian (points=8192 (128x64)  F32)
Trying gencon2
Written tas_Amon_CanESM5-CanOE_historical_r1i1p2f1_gn_185001-201412.gencon2_weights.nc
1 input data file(s) to regrid
Written tas_Amon_CanESM5-CanOE_historical_r1i1p2f1_gn_185001-201412.con2.yearmean.nc

---- tas Amon CanESM5_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CCCma.CanESM5.historical.r1i1p1f1.Amon.tas.gn.v20190429 (1 nc file(s))
Input data grid type is gaussian (points=8192 (128x64)  F32)
Trying gencon2
Written tas_Amon_CanESM5_historical_r1i1p1f1_gn_185001-201412.gencon2_weights.nc
1 input data file(s) to regrid
Written tas_Amon_CanESM5_historical_r1i1p1f1_gn_185001-201412.con2.yearmean.nc

---- tas Amon CanESM5_r1i1p2f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.CCCma.CanESM5.historical.r1i1p2f1.Amon.tas.gn.v20190429 (1 nc file(s))
Input data grid type is gaussian (points=8192 (128x64)  F32)
Trying gencon2
Written tas_Amon_CanESM5_historical_r1i1p2f1_gn_185001-201412.gencon2_we

Written tas_Amon_INM-CM5-0_historical_r1i1p1f1_gr1_185001-194912.gencon2_weights.nc
2 input data file(s) to regrid
Written 2 files, inc tas_Amon_INM-CM5-0_historical_r1i1p1f1_gr1_185001-194912.con2.yearmean.nc

---- tas Amon IPSL-CM5A2-INCA_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.IPSL.IPSL-CM5A2-INCA.historical.r1i1p1f1.Amon.tas.gr.v20200729 (1 nc file(s))
Input data grid type is lonlat (points=9216 (96x96))
Trying gencon2
Written tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_185001-201412.gencon2_weights.nc
1 input data file(s) to regrid
Written tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_185001-201412.con2.yearmean.nc

---- tas Amon IPSL-CM6A-LR_r1i1p1f1 historical -yearmean ----
1 input dir: CMIP6.CMIP.IPSL.IPSL-CM6A-LR.historical.r1i1p1f1.Amon.tas.gr.v20180803 (1 nc file(s))
Input data grid type is lonlat (points=20592 (144x143))
Trying gencon2
Written tas_Amon_IPSL-CM6A-LR_historical_r1i1p1f1_gr_185001-201412.gencon2_weights.nc
1 input data file(s) to regr

In [7]:
! date

Fri Jan 28 14:08:37 +08 2022
